# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [ ]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [4]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(frog))

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/solutions/government',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/cla

In [ ]:
get_links("https://anthropic.com")

In [ ]:
get_links("https://cursos.frogamesformacion.com")

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [8]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://anthropic.com"))

In [9]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [10]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [ ]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

In [11]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Anthropic", "https://anthropic.com")

In [ ]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [12]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [14]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Página de Clientes', 'url': 'https://www.anthropic.com/customers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}]}


# Folleto de Anthropic

---

## Acerca de Anthropic

Anthropic es una empresa pública de beneficios enfocada en la investigación y desarrollo de inteligencia artificial (IA) que prioriza la seguridad y la confiabilidad. Con sede en San Francisco, nuestro equipo multicultural incluye expertos en diversas disciplinas, como física, aprendizaje automático, política pública y negocios. Creemos que la IA tiene el potencial de transformar el mundo y trabajamos para garantizar que su evolución beneficie a la humanidad.

![Claude](https://example.com/claude_image.png)  *(Aquí se puede incluir una imagen del producto Claude)*

---

## Nuestros Productos

### Claude

**Claude** es nuestra línea de modelos de IA, incluidos **Claude Opus 4.1** y **Sonnet 4**, diseñados para ser interpretables, confiables y manejables. Están disponibles para diversas aplicaciones en empresas, educación, servicios financieros y más.

- **API**: Permite crear aplicaciones personalizadas utilizando Claude.
- **Herramientas**: Enfocadas en el desarrollo de agentes de IA y soporte al cliente.

---

## Cultura del Trabajo

### Nuestros Valores

1. **Actuar por el bien global**: Tomamos decisiones que maximizan resultados positivos a largo plazo para la humanidad.
2. **Mantener luz y sombra**: Equilibramos los riesgos y beneficios del desarrollo de IA.
3. **Ser buenos con nuestros usuarios**: Cultivamos la generosidad y amabilidad en todas nuestras interacciones.
4. **Impulsar una competencia en seguridad**: Buscamos inspirar a otros en la industria a lograr estándares más altos de seguridad en IA.
5. **Hacer lo simple que funciona**: Optamos por soluciones efectivas y prácticas.
6. **Ser útiles, honestos y inofensivos**: Fomentamos un ambiente de confianza y colaboración.
7. **Poner la misión primero**: Nuestro propósito compartido guía todas nuestras acciones y decisiones.

---

## Oportunidades de Carrera

### ¿Por qué trabajar en Anthropic?

Ofrecemos un entorno de trabajo inclusivo y colaborativo, con beneficios diseñados para apoyar la salud y el bienestar de nuestros empleados. Algunas de nuestras ofertas incluyen:

- **Salud y Bienestar**: Seguro médico completo, beneficios de fertilidad inclusivos y 22 semanas de licencia parental pagada.
- **Compensación**: Salarios competitivos y paquetes de acciones, además de planes de jubilación robustos.
- **Beneficios adicionales**: Estipendios para educación, bienestar personal, y soporte para reubicación.
- **Comida**: Ofrecemos comidas y refrigerios en la oficina.

### Carreras en Anthropic

Estamos buscando individuos apasionados que deseen contribuir a un futuro seguro y responsable en IA. Consulta nuestras [oportunidades laborales](https://anthropic.com/careers) abiertas.

---

## Educación y Aprendizaje

### Anthropic Academy

Nuestra academia ofrece recursos y cursos para ayudar a los futuros profesionales a aprender sobre desarrollo de IA utilizando Claude. Puedes acceder a estos recursos en nuestra plataforma y avanzar en tu carrera en este campo prometedor.

---

## Conclusión

En Anthropic, estamos comprometidos a guiar de manera segura el progreso de la inteligencia artificial. Únete a nosotros en esta misión y contribuye a construir un futuro donde la tecnología beneficie a toda la humanidad.

Para más información, visita nuestro sitio web en [Anthropic.com](https://anthropic.com).

--- 

*¿Tienes preguntas? ¡No dudes en contactarnos!*

---

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.